In [7]:
!git clone https://github.com/jan1na/Neural-Cellular-Automata.git

%cd Neural-Cellular-Automata

Cloning into 'Neural-Cellular-Automata'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 15 (delta 5), reused 9 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (15/15), 8.52 KiB | 8.52 MiB/s, done.
Resolving deltas: 100% (5/5), done.
eval.ipynb		  __pycache__  train_cnn.ipynb	train_utils.py
Neural-Cellular-Automata  README.md    train_nca.ipynb
/content/Neural-Cellular-Automata/Neural-Cellular-Automata
/content/Neural-Cellular-Automata/Neural-Cellular-Automata/Neural-Cellular-Automata
/content/Neural-Cellular-Automata/Neural-Cellular-Automata/Neural-Cellular-Automata


In [8]:
!pip install medmnist torch torchvision tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import DataLoader
from medmnist import PathMNIST
import medmnist
from tqdm import tqdm
from train_utils import train, evaluate
from models import NCA

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
medmnist.INFO['pathmnist']['download'] = True

transform = transforms.Compose([transforms.ToTensor()])

train_dataset = PathMNIST(split='train', transform=transform, download=True)
val_dataset = PathMNIST(split='val', transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

NUM_CLASSES = 9

100%|██████████| 206M/206M [00:11<00:00, 18.5MB/s]


In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = NCA().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

best_acc = 0
model_path = '/content/drive/MyDrive/NCA/best_nca_pathmnist.pth'

for epoch in range(1, 51):
    train_loss, train_acc = train(model, train_loader, optimizer, criterion, device)
    val_loss, val_acc = evaluate(model, val_loader, criterion, device)
    print(f"Epoch {epoch:02d}: Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")

    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), model_path)
        print(f"New best model saved to {model_path}")

    if val_acc > 0.90:
        print("Converged (validation accuracy > 90%)")
        break

print("Training completed.")

Epoch 01: Train Acc: 0.5657, Val Acc: 0.6697
🔥 New best model saved to /content/drive/MyDrive/NCA/best_nca_pathmnist.pth
Epoch 02: Train Acc: 0.7387, Val Acc: 0.8176
🔥 New best model saved to /content/drive/MyDrive/NCA/best_nca_pathmnist.pth
Epoch 03: Train Acc: 0.8224, Val Acc: 0.8399
🔥 New best model saved to /content/drive/MyDrive/NCA/best_nca_pathmnist.pth
Epoch 04: Train Acc: 0.8650, Val Acc: 0.8899
🔥 New best model saved to /content/drive/MyDrive/NCA/best_nca_pathmnist.pth
Epoch 05: Train Acc: 0.8894, Val Acc: 0.9092
🔥 New best model saved to /content/drive/MyDrive/NCA/best_nca_pathmnist.pth
Converged (validation accuracy > 90%)
🎉 Training completed.
